# **Introduction to PSScriptAnalyzer** 🚀

## **What is PSScriptAnalyzer?**

PSScriptAnalyzer is a static code checker for PowerShell modules and scripts. PSScriptAnalyzer can be used to check the quality of PowerShell by running a set of rules. It is possible to use built-in rules based on PowerShell best practices identified by PowerShell Team and the community. It is also possible to write custom rules or/and combine the two. It generates DiagnosticResults (errors and warnings) to inform users about potential code defects and suggests possible solutions for improvements.

## **Why Use PSScriptAnalyzer?**

One of the key reasons to use PSScriptAnalyzer is to minimize technical debt. By enforcing code quality and adhering to best practices, you can prevent the accumulation of issues that could lead to maintenance challenges later on.

## **How to Get Started**

### **Clone git repo**

```bash 
    git clone https://github.com/qriticual/PSScriptAnalyzer.git
```

### **Installation**
To install PSScriptAnalyzer, use the following PowerShell command:

```powershell
    Install-Module -Name PSScriptAnalyzer -Force
```

We also need to install the `PowerShell` extension in VSCode

### **Setting Up PSScriptAnalyzer in VSCode**

To ensure a seamless experience within Visual Studio Code (VSCode), follow these steps:

1. Open the `.vscode/settings.json` file in your project.

2. Add the following lines to the file:

```json
{
    "powershell.scriptAnalysis.enable": true,
    "powershell.scriptAnalysis.settingsPath": "PSScriptAnalyzerSettings.psd1"
}
```


This will ensure that you:
- Enable scriptAnalysis for your IDE. 
- Specify the use of settings defined in the PSScriptAnalyzerSettings.psd1 file.

Create a new file named `PSScriptAnalyzerSettings.psd1` in your project's root directory. 

Here's a simple content for `PSScriptAnalyzerSettings.psd1`:


In [ ]:
# PSScriptAnalyzerSettings.psd1
@{
    IncludeDefaultRules = $true
}

Create a new .ps1 file and try it out. One default rule is that $null should be on the left hand side of equality comparison.

In [ ]:
if ($he -eq $null) { Write-Output 'Hello World' 
}

### **Use built-in rules**

PSScriptAnalyzer comes with predefined rules that can be included or excluded. [Learn more about built-in rules.](https://learn.microsoft.com/en-us/powershell/utility-modules/psscriptanalyzer/rules/readme?view=ps-modules)

To use some built-in rules, update your `PSScriptAnalyzerSettings.psd1`:

In [ ]:
@{
    Rules               = @{
        PSPlaceOpenBrace           = @{
            Enable             = $true
            OnSameLine         = $true
            NewLineAfter       = $true
            IgnoreOneLineBlock = $true
        }
        PSPlaceCloseBrace          = @{
            Enable             = $true
            NoEmptyLineBefore  = $true
            IgnoreOneLineBlock = $true
            NewLineAfter       = $false
        }
        PSAlignAssignmentStatement = @{
            Enable         = $true
            CheckHashtable = $true
        }
    }
    IncludeDefaultRules = $true
    IncludeRules        = @(
        # Default rules
        'PSPlaceOpenBrace'
        'PSPlaceCloseBrace'
        'PSAlignAssignmentStatement'
    )
}

See the 'yellow' marked errors in the syntax and recommended actions to resolve the issues. Mark code block and press shift+alt+f to autofix syntax errors.

In [ ]:
if($he -eq $null){
    Write-Output 'Hello World'}   

### **Use custom rules rules**

Create a CustomPSScriptAnalyzerRules.psm1 file with a custom rule. For example, a rule to replace Invoke-Something with Invoke-SomethingElse:

[Learn more about custom rules.](https://learn.microsoft.com/en-us/powershell/utility-modules/psscriptanalyzer/create-custom-rule?view=ps-modules)

In [ ]:
function Invoke-SomethingRule {
    <#
    .DESCRIPTION
        Custom rule to warn against using Invoke-Something and suggest Invoke-SomethingElse.
    #>

    param(
        [System.Management.Automation.Language.CommandAst]$ast
    )

    if ($ast.GetCommandName() -eq 'Invoke-Something') {
        Write-Verbose 'hello'
        [int]$startLineNumber = $ast.Extent.StartLineNumber
        [int]$endLineNumber = $ast.Extent.EndLineNumber
        [int]$startColumnNumber = $ast.Extent.StartColumnNumber
        [int]$endColumnNumber = $ast.Extent.EndColumnNumber
        [string]$correction = 'Invoke-SomethingElse'
        # [string]$file = $MyInvocation.MyCommand.Definition
        [string]$optionalDescription = 'Useful but optional description text'
        $objParams = @{
            TypeName     = 'Microsoft.Windows.PowerShell.ScriptAnalyzer.Generic.CorrectionExtent'
            ArgumentList = $startLineNumber, $endLineNumber, $startColumnNumber,
            $endColumnNumber, $correction, $optionalDescription
        }
        $correctionExtent = New-Object @objParams
        $suggestedCorrections = New-Object System.Collections.ObjectModel.Collection[$($objParams.TypeName)]
        $suggestedCorrections.add($correctionExtent) | Out-Null

        $diagnosticRecord = [Microsoft.Windows.PowerShell.ScriptAnalyzer.Generic.DiagnosticRecord]@{
            Message              = "Don't use Invoke-Something; use Invoke-SomethingElse instead."
            Extent               = $ast.Extent
            RuleName             = 'Invoke-SomethingRule'
            Severity             = 'Warning'
            SuggestedCorrections = $suggestedCorrections
        }

        return $diagnosticRecord
    }
}

Export-ModuleMember -Function Invoke-SomethingRule

Update your settings file to include the custom rule:

In [ ]:
@{
    Rules               = @{
        PSPlaceOpenBrace           = @{
            Enable             = $true
            OnSameLine         = $true
            NewLineAfter       = $true
            IgnoreOneLineBlock = $true
        }
        PSPlaceCloseBrace          = @{
            Enable             = $true
            NoEmptyLineBefore  = $true
            IgnoreOneLineBlock = $true
            NewLineAfter       = $false
        }
        PSAlignAssignmentStatement = @{
            Enable         = $true
            CheckHashtable = $true
        }
    }
    IncludeDefaultRules = $true
    CustomRulePath      = 'CustomPSScriptAnalyzerRules.psm1'
    IncludeRules        = @(
        # Default rules
        'PSPlaceOpenBrace'
        'PSPlaceCloseBrace'
        'PSAlignAssignmentStatement'
        # Custom rules
        'Invoke-SomethingRule'
    )
}

Test the new custom rule:

In [ ]:
Invoke-Something

### **Use it from the terminal/as a script**

PSScriptAnalyzer can also be used from the terminal and not only in the IDE. This offers opportunities like scanning the whole repository for warnings at once and using it in a pipeline to scan for errors.

To use it, execute the `Invoke-ScriptAnalyzer` command from the `PSScriptAnalyzer` module. For reference on how to use it, visit [Using ScriptAnalyzer.](https://learn.microsoft.com/en-us/powershell/utility-modules/psscriptanalyzer/using-scriptanalyzer?view=ps-modules)

In [ ]:
# built-inpresets 'DSC', 'PSGallery', 'CodeFormatting'
# Invoke-ScriptAnalyzer -Path './scriptAnalyzer.ps1' -Settings 'CodeFormatting'
Invoke-ScriptAnalyzer -Path './test.ps1' -Settings './PSScriptAnalyzerSettings.psd1'

Use it to scan the whole repository:

In [ ]:
$ps1files = Get-ChildItem -Filter '*.ps1' -Recurse

foreach ($file in $ps1files) {
    Write-Output "`n*** $($file.Directory.Name):`n"
    Invoke-ScriptAnalyzer -Path $file -Settings '.\PSScriptAnalyzerSettings.psd1'
}

### **Auto fixes as a script**

To apply fixes on one single file:
```powershell
    Invoke-ScriptAnalyzer -Path scriptAnalyzer.ps1 -Fix
```

To apply fixes on every .ps1 file in your current folder:
```powershell
    $ps1files = Get-ChildItem -Filter '*.ps1' -Recurse

    foreach ($file in $ps1files) {
        Write-Output "`n*** $($file.Directory.Name):`n"
        Invoke-ScriptAnalyzer -Path $file -Settings '.\PSScriptAnalyzerSettings.psd1' -Fix
    }
```

### **Auto fixes VSCode short command**
alt + shift + f 

### **Additional custom rules**

In [ ]:
function Invoke-SomethingRule {
    <#
    .DESCRIPTION
        Custom rule to warn against using Invoke-Something and suggest Invoke-SomethingElse.
    #>

    param(
        [System.Management.Automation.Language.CommandAst]$ast
    )

    if ($ast.GetCommandName() -eq 'Invoke-Something') {
        Write-Verbose 'hello'
        [int]$startLineNumber = $ast.Extent.StartLineNumber
        [int]$endLineNumber = $ast.Extent.EndLineNumber
        [int]$startColumnNumber = $ast.Extent.StartColumnNumber
        [int]$endColumnNumber = $ast.Extent.EndColumnNumber
        [string]$correction = 'Invoke-SomethingElse'
        # [string]$file = $MyInvocation.MyCommand.Definition
        [string]$optionalDescription = 'Useful but optional description text'
        $objParams = @{
            TypeName     = 'Microsoft.Windows.PowerShell.ScriptAnalyzer.Generic.CorrectionExtent'
            ArgumentList = $startLineNumber, $endLineNumber, $startColumnNumber,
            $endColumnNumber, $correction, $optionalDescription
        }
        $correctionExtent = New-Object @objParams
        $suggestedCorrections = New-Object System.Collections.ObjectModel.Collection[$($objParams.TypeName)]
        $suggestedCorrections.add($correctionExtent) | Out-Null

        $diagnosticRecord = [Microsoft.Windows.PowerShell.ScriptAnalyzer.Generic.DiagnosticRecord]@{
            Message              = "Don't use Invoke-Something; use Invoke-SomethingElse instead."
            Extent               = $ast.Extent
            RuleName             = 'Invoke-SomethingRule'
            Severity             = 'Warning'
            SuggestedCorrections = $suggestedCorrections
        }

        return $diagnosticRecord
    }
}

Export-ModuleMember -Function Invoke-SomethingRule

function Test-UnaryOperatorRule { 
    <#
    .DESCRIPTION
        Custom rule to warn against using spaces when using unary operators and suggest removing them.
    #>
    param(
        [System.Management.Automation.Language.UnaryExpressionAst]$ast
    )

    $pattern = '(\+\+|--)\s|\s(\+\+|--)'
    if ($ast.Extent.Text -match $pattern) {
        [int]$startLineNumber = $ast.Extent.StartLineNumber
        [int]$endLineNumber = $ast.Extent.EndLineNumber
        [int]$startColumnNumber = $ast.Extent.StartColumnNumber
        [int]$endColumnNumber = $ast.Extent.EndColumnNumber
        [string]$correction = $ast.Extent.Text -replace '\s', ''
        # [string]$file = $MyInvocation.MyCommand.Definition
        [string]$optionalDescription = "Don't use spaces when using unary operators; remove them instead"

        $objParams = @{
            TypeName     = 'Microsoft.Windows.PowerShell.ScriptAnalyzer.Generic.CorrectionExtent'
            ArgumentList = $startLineNumber, $endLineNumber, $startColumnNumber,
            $endColumnNumber, $correction, $optionalDescription
        }
        $correctionExtent = New-Object @objParams
        $suggestedCorrections = New-Object System.Collections.ObjectModel.Collection[$($objParams.TypeName)]
        $suggestedCorrections.add($correctionExtent) | Out-Null
    
        $diagnosticRecord = [Microsoft.Windows.PowerShell.ScriptAnalyzer.Generic.DiagnosticRecord]@{
            Message              = "Don't use spaces when using unary operators; remove them instead."
            Extent               = $ast.Extent
            RuleName             = 'Test-UnaryOperatorRule'
            Severity             = 'Warning'
            SuggestedCorrections = $suggestedCorrections
        }
    
        return $diagnosticRecord
    }
}   

Export-ModuleMember -Function Test-UnaryOperatorRule

function Test-FunctionCasing {
    [CmdletBinding()] [OutputType([PSCustomObject[]])] param (
        [Parameter(Mandatory)] [ValidateNotNullOrEmpty()] [System.Management.Automation.Language.ScriptBlockAst]$ScriptBlockAst
    )

    process {
        try {
            $functions = $ScriptBlockAst.FindAll( 
                { 
                    $args[0] -is [System.Management.Automation.Language.FunctionDefinitionAst] -and
                    $args[0].Name -cmatch '[A-Z]{2,}' 
                }, $true ) 
            foreach ( $function in $functions ) { 
                [PSCustomObject]@{ 
                    Message  = 'Avoid function names with adjacent caps in their name'
                    Extent   = $function.Extent 
                    RuleName = $PSCmdlet.MyInvocation.InvocationName 
                    Severity = 'Warning' 
                }
            }
        } catch {
            $PSCmdlet.ThrowTerminatingError( $_ )
        }
    }
}

Export-ModuleMember -Function Test-FunctionCasing 

### **Evolved settingsfile**

In [ ]:
@{
    Rules               = @{
        PSPlaceOpenBrace           = @{
            Enable             = $true
            OnSameLine         = $true
            NewLineAfter       = $true
            IgnoreOneLineBlock = $true
        }
        PSPlaceCloseBrace          = @{
            Enable             = $true
            NoEmptyLineBefore  = $true
            IgnoreOneLineBlock = $true
            NewLineAfter       = $false
        }
        PSUseConsistentIndentation = @{
            Enable          = $true
            IndentationSize = 4
            Kind            = 'space'
        }
        PSUseConsistentWhitespace  = @{
            Enable                                  = $true
            CheckInnerBrace                         = $true
            CheckOpenBrace                          = $true
            CheckOpenParen                          = $true
            CheckOperator                           = $true
            CheckSeparator                          = $true
            CheckPipe                               = $true
            CheckPipeForRedundantWhitespace         = $true
            CheckParameter                          = $true
            IgnoreAssignmentOperatorInsideHashTable = $true
        }
        PSAlignAssignmentStatement = @{
            Enable         = $true
            CheckHashtable = $true
        }
    }
    CustomRulePath      = 'CustomPSScriptAnalyzerRules.psm1'
    IncludeDefaultRules = $true
    IncludeRules        = @(
        # Default rules
        'PSPlaceOpenBrace'
        'PSPlaceCloseBrace'
        'PSUseConsistentIndentation'
        'PSUseConsistentWhitespace'
        'PSAlignAssignmentStatement'

        # Custom rules
        'Invoke-SomethingRule'
        'Test-UnaryOperatorRule'
        'Test-FunctionCasing'
    )
}

For additional rules and recommendations from the PowerShell team and community, refer to [Rules and Recommendations.](https://learn.microsoft.com/en-us/powershell/utility-modules/psscriptanalyzer/rules-recommendations?view=ps-modules)